# Importing and processing raw data from the <br>Proteomic Data Commons ([PDC](https://proteomic.datacommons.cancer.gov/pdc/))



### Overview
The **Proteomic Data Commons** ([PDC](https://proteomic.datacommons.cancer.gov/pdc/)) is a data repository within the [NCI Cancer Research Data Commons (CRDC)](https://datacommons.cancer.gov/) and provides access to curated and standardized *proteomic data* along with biospecimen and clinical metadata. The goal of this notebook is to faciliate *automated data download and streamlined analysis* of raw proteomic data imported from the PDC using the [`FragPipe`](https://fragpipe.nesvilab.org/) proteomics pipeline.

#### Caveat
This notebook is a **proof of principle** and works for *TMT* or *iTRAQ* raw data from the PDC. It may not be robust to different types of inputs and raw data files, and implements only minimal error checking. With knowledge of input requirements for `FragPipe`, this notebook can be modified to support other types of data imports from the PDC.

### Using this notebook
When data files are selected at the PDC and exported to [Terra](http://app.terra.bio), a `file` table is automatically created and populated in the chosen Terra workspace. The Python code in this notebook transfers data files pointed to by [DRS URIs](https://support.terra.bio/hc/en-us/articles/360039330211-Overview-Interoperable-data-GA4GH-DRS-URIs-) in the `file` table to the Google bucket for the workspace. In addition to transferring raw data files, this script also creates the annotation and manifest files needed to run `FragPipe` on the raw data files using the `panoply_fragpipe_search` workflow from the [PANOPLY](https://github.com/broadinstitute/PANOPLY) proteogenomic data analysis platform.

Step-by-step process for importing and processing raw proteomics data files:
1. Start by navigating to the [PDC](https://proteomic.datacommons.cancer.gov/pdc/). 
2. Browse through the proteomics raw data available there and select files to import and process on Terra. Analyzing TMT or iTRAQ raw data using `FragPipe` requires `mzML` files. On the PDC, `mzML` files are listed in the `Processed Mass Spectra (Open Standard)` data category.
3. Export file manifest for the chosen files using the `PFB` button. This will connect to Terra and prompt for a workspace to put locate the `file` manifest table.
4. Copy the `PDC_Direct_Data_Import` notebook to the workspace with the `file` manifest table and run all the code blocks in sequence. Running the code blocks will:
    1. Download all the files listed in the table to the `fragpipe` directory in the workspace bucket.
    2. Organize files into subdirectories--one subdirectory for each TMT/iTRAQ plex, including all fractions for that plex.
    3. Create annotation files for each TMT/iTRAQ plex to provide sample IDs.
    4. Generate a `FragPipe` manifest file for processing all the data files.
5. Once the notebook has been successfully run, create a `FragPipe` [workflow](https://fragpipe.nesvilab.org/docs/tutorial_fragpipe_workflows.html) using the `FragPipe` graphical user interface. Workflows can also be directly [downloaded](https://github.com/Nesvilab/FragPipe/tree/master/MSFragger-GUI/workflows) and/or edited as needed. Upload the workflow to the workspace bucket.
6. Obtain a relevant (fasta) sequence database and upload to the workspace bucket. Sequence databases can be downloaded using the `FragPipe` graphical user interface, or copied from other workspaces.
7. If not already present, import the `panoply_fragpipe_search` method from the [FireCloud Method Repository](https://portal.firecloud.org/?return=terra#methods).
8. Set up inputs to the `panoply_fragpipe_search` workflow:
    1. Point `database` to the sequence database in the workspace bucket (from Step 6).
    2. `files_folder` is the bucket address of the `fragpipe` directory created to host all the downloaded files.
    3. The location of the manifest file `fragpipe-manifest.fp-manifest` created by the notebook should be filled into `fragpipe_manifest`.
    4. The bucket location of the workflow file uploaded in Step 5 goes in the `fragpipe_workflow` input slot.
9. Run the workflow and download the output `zip` files.

**NB:** The Python code below is interspersed with comments to describe its function and logic.

### Import libraries needed

In [1]:
## Libraries

import os
import requests
import json
import hashlib
import gzip
import shutil
import pandas as pd
from terra_notebook_utils import drs, table

### Set target directory on the Workspace Bucket
In the workspace bucket, files are put into the `fragpipe` directory. In this directory, subdirectories are created for each experiment in order to support TMT workflows using FragPipe. All files in the `file` table are transferred. In each subdirectory representing a TMT/iTRAQ plex, an `annotation.txt` file is created to map the TMT/iTRAQ channels to sample or aliquot IDs. Additionally, a `fragpipe-manifest.fp-manifest` manifest file is create in the root directory of the workspace bucket to set up the entire analysis.

The code section below establishes various global variables for data transfer, annotation and setup for FragPipe analysis. 

In [2]:
## Global variables

# Terra workspace related variables
target_dir = "fragpipe"
bucket = os.environ.get ("WORKSPACE_BUCKET")
target_path = os.path.join (bucket, target_dir)

# fragpipe related variables
annot_file = "annotation.txt"
manifest_file = "fragpipe-manifest.fp-manifest"
manifest_df = pd.DataFrame(columns = ['Path', 'Experiment', 'Bioreplicate', 'DataType'])

# URL for PDC API calls
pdc_url = 'https://pdc.cancer.gov/graphql'

# query for experimental design data from PDC
query = '''{
  studyExperimentalDesign(pdc_study_id: "%s" acceptDUA: true) {
    acquisition_type
    analyte
    experiment_number
    experiment_type
    number_of_fractions
    pdc_study_id
    plex_dataset_name
    study_id
    study_run_metadata_id
    study_run_metadata_submitter_id
    study_submitter_id
    label_free{aliquot_submitter_id}
    itraq_113{aliquot_submitter_id}
    itraq_114{aliquot_submitter_id}
    itraq_115{aliquot_submitter_id}
    itraq_116{aliquot_submitter_id}
    itraq_117{aliquot_submitter_id} 
    itraq_118{aliquot_submitter_id}
    itraq_119{aliquot_submitter_id}
    itraq_121{aliquot_submitter_id}
    tmt_126{aliquot_submitter_id}
    tmt_127n{aliquot_submitter_id} 
    tmt_127c{aliquot_submitter_id}
    tmt_128n{aliquot_submitter_id}
    tmt_128c{aliquot_submitter_id}
    tmt_129n{aliquot_submitter_id}
    tmt_129c{aliquot_submitter_id} 
    tmt_130n{aliquot_submitter_id} 
    tmt_130c{aliquot_submitter_id}
    tmt_131{aliquot_submitter_id} 
    tmt_131c{aliquot_submitter_id} 
    tmt_132n{aliquot_submitter_id}
    tmt_132c{aliquot_submitter_id} 
    tmt_133n{aliquot_submitter_id}
    tmt_133c{aliquot_submitter_id} 
    tmt_134n{aliquot_submitter_id} 
    tmt_134c{aliquot_submitter_id}
    tmt_135n{aliquot_submitter_id}
  }
}'''

# variables for managing input table, experiment design and annotations
file_rows = table.list_rows ("file")
expt_design = {}
plex_annot = {}

### Functions
Define functions to streamline code in the later blocks and to illustrate the flow of the script in a more conceptual manner.

In [3]:
## Functions to streamline code

def md5 (fname):
  # generate MD5 checksum for given file
  # from https://stackoverflow.com/questions/3431825/generating-an-md5-checksum-of-a-file
  hash_md5 = hashlib.md5()
  with open(fname, "rb") as f:
    for chunk in iter(lambda: f.read(4096), b""):
      hash_md5.update(chunk)
  return hash_md5.hexdigest()

def download_file (url, f, checksum):
  ret_file = f
  # download file and check MD5 checksum
  if os.path.exists (f):
    os.remove (f)     # remove previous files, if any
  drs.copy (url, f)
  if (md5(f) != checksum):
    sys.exit ('** ERROR: File download failed.')
  # unzip file it is zipped (specifically for mzML.gz files)
  fname, ext = os.path.splitext (f)
  if ext == ".gz":
    ret_file = fname  # return file name without .gz extension
    with gzip.open(f, 'rb') as f_in, open(fname, 'wb') as f_out:
      shutil.copyfileobj(f_in, f_out)
    
  return ret_file
  
def transfer_to_bucket (subdir, f, root=target_path):
  # transfer given file to specified subdirectory in the workspace bucket
  os.system ("gsutil -q cp " + f + " " + os.path.join (root, subdir, f))

def download_expt_design (study):
  response = requests.post(pdc_url, json={'query': query % study})
  if(response.ok):
    jData = json.loads(response.content)
    study_design = jData['data']['studyExperimentalDesign']
    expt_design[study] = study_design
  else:   # If response code is not ok (200)
    response.raise_for_status()
  return study_design

def create_annotation (des, plx, subdir, a_file):
  print ('** Writing annotation file ' + os.path.join (subdir, a_file))
  if os.path.exists (a_file):
    os.remove (a_file)    # remove previous files, if any
  expt_type = des[plx]['experiment_type']
  if (expt_type.startswith ('TMT') or expt_type.startswith ('iTRAQ')) and des[plx]['acquisition_type'] == "DDA": 
    # for TMT/iTRAQ experiments
    # get TMT aliquot IDs and create annotation file
    aliquots = {key: val[0]['aliquot_submitter_id'] 
                for key, val in des[plx].items() 
                  if (key.startswith('tmt') or key.startswith('itraq')) and val != None}
    with open (a_file, "a") as f:
      for i, k in enumerate (aliquots.keys()):
        aliquot = aliquots[k].replace (" ", "")
        if 'Pool' in aliquot or 'pool' in aliquot:
          aliquot = 'pool' + str (plx)
        ch = k.split ('tmt_')[1].upper()
        print (ch, aliquot, file=f)
    transfer_to_bucket (subdir, a_file)
  else:
    # To-Do: Implement options for label-free, DIA
    sys.exit ('** ERROR: Only TMT/iTRAQ DDA experiments supported at this time.')

def write_manifest (df, m_file):
  print ('** Writing manifest file ' + m_file)
  if os.path.exists (m_file):
    os.remove (m_file)    # remove previous files, if any
  df.sort_values (by=['Experiment']).to_csv (m_file, header=None, index=None, sep='\t', mode='a')
  transfer_to_bucket ('', m_file, root=bucket)

### Transfer the files
Direct copy to the bucket using `drs.copy_to_bucket` does not work and results in an error -- download file to local disk and use `gsutil` to copy to the workspace bucket. As files are downloaded, use the PDC `graphql` API to get experimental design to create the annotation files. Also assemble and output the manifest.

In [4]:
# NOTE: Downloading the DRS_URL file may require the user to login to NCI CRDC Framework Services
#       Set up access (if needed) using the EXTERNAL INDENTITIES tab in the USER PROFILE on Terra 
#         before running this code
for row in file_rows:
  # extract relevant fields from row
  drs_url = row.attributes ["pfb:object_id"]
  file_name = row.attributes ["pfb:file_name"]
  expt_subdir = row.attributes ["pfb:study_run_metadata_submitter_id"]
  pdc_study_id = row.attributes ["pfb:pdc_study_id"]
  md5_checksum = row.attributes ["pfb:file_md5sum"]

  # download file and transfer to workspace bucket
  file_name = download_file (drs_url, file_name, md5_checksum)
  transfer_to_bucket (expt_subdir, file_name)
  os.remove (file_name)  # delete local file
    
  # add to manifest table
  manifest_df = manifest_df.append ({'Path': os.path.join ("/root/fragpipe/data", expt_subdir, file_name),
                                     'Experiment': expt_subdir, 'Bioreplicate': '', 'DataType': 'DDA'},
                                    ignore_index=True)
  # download experiment design metadata (if needed -- keep previous downloads)
  if pdc_study_id in expt_design.keys():
    design = expt_design[pdc_study_id]
  else:
    design = download_expt_design (pdc_study_id)

  # get plex details and create required annotation files 
  plex = -1
  for i in range (len (design)):
    if design[i]['study_run_metadata_submitter_id'] == expt_subdir:
      plex = i
      break

  if (plex_annot.get(plex) == None): 
    # current plex has not already been annotated
    plex_annot[plex] = True
    create_annotation (design, plex, expt_subdir, annot_file)

# write out manifest file
write_manifest (manifest_df, manifest_file)


/home/jupyter/PDC-PFB-Test/edit/02CPTAC_LUAD_W_BI_20180518_KR_f02.mzML.gz


** Writing annotation file S046-1-2/annotation.txt
/home/jupyter/PDC-PFB-Test/edit/01CPTAC_LUAD_W_BI_20180515_KR_f04.mzML.gz


** Writing annotation file S046-1-1/annotation.txt
/home/jupyter/PDC-PFB-Test/edit/02CPTAC_LUAD_W_BI_20180518_KR_f01.mzML.gz


/home/jupyter/PDC-PFB-Test/edit/01CPTAC_LUAD_W_BI_20180515_KR_f03.mzML.gz


/home/jupyter/PDC-PFB-Test/edit/01CPTAC_LUAD_W_BI_20180515_KR_f02.mzML.gz


...

/home/jupyter/PDC-PFB-Test/edit/02CPTAC_LUAD_W_BI_20180518_KR_f05.mzML.gz


/home/jupyter/PDC-PFB-Test/edit/02CPTAC_LUAD_W_BI_20180518_KR_f03.mzML.gz


/home/jupyter/PDC-PFB-Test/edit/01CPTAC_LUAD_W_BI_20180515_KR_f01.mzML.gz


/home/jupyter/PDC-PFB-Test/edit/02CPTAC_LUAD_W_BI_20180518_KR_f04.mzML.gz


** Writing manifest file fragpipe-manifest.fp-manifest
